# Trusted Zone — Documents Processing

This notebook handles the **document processing** step for the Trusted Zone of our data pipeline.  
Its primary goal is to:

1. **Load recipe IDs** that have images (from the images processing step)
2. **Stream-filter the recipe documents** to keep only those with images
3. **Use multipart uploads** to handle large JSONL files efficiently
4. **Generate a processing report** for audit and monitoring

This notebook works in conjunction with `images.ipynb` to ensure data integrity in the Trusted Zone.


## 1. Setup and Configuration


In [6]:
import os, io, json, re
from pathlib import PurePosixPath
from datetime import datetime, timezone
from typing import Dict, List, Set, Iterable

import boto3
from botocore.config import Config
from botocore.exceptions import ClientError
from dotenv import load_dotenv

load_dotenv()

# S3 / MinIO Configuration
MINIO_USER     = os.getenv("MINIO_USER")
MINIO_PASSWORD = os.getenv("MINIO_PASSWORD")
MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT")

session = boto3.session.Session(
    aws_access_key_id=MINIO_USER,
    aws_secret_access_key=MINIO_PASSWORD,
    region_name="us-east-1"
)
s3 = session.client(
    "s3",
    endpoint_url=MINIO_ENDPOINT,
    config=Config(signature_version="s3v4", s3={"addressing_style": "path"})
)

# Paths and Buckets
FORM_BUCKET         = "formatted-zone"
FORM_DOCS_KEY       = "documents/recipes.jsonl"

TRUST_BUCKET        = "trusted-zone"
TRUST_DOCS_KEY      = "documents/recipes.jsonl"
TRUST_REPORT_PREFIX = "reports"

# Input file from images processing
RECIPE_IDS_FILE = "recipe_ids_with_images.json"

# Behavior flags
DRY_RUN   = False
OVERWRITE = True

def utc_ts():
    return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H-%M-%SZ")


## 2. Load Recipe IDs from Images Processing

We load the recipe IDs that have images, which were generated by the `images.ipynb` notebook. This creates the coupling between the two processing steps while maintaining clean separation.


In [7]:
def load_recipe_ids_with_images() -> Set[str]:
    """Load recipe IDs that have images from the images processing step."""
    try:
        with open(RECIPE_IDS_FILE, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        recipe_ids = set(data.get('recipe_ids_with_images', []))
        print(f"[INFO] Loaded {len(recipe_ids)} recipe IDs with images")
        print(f"[INFO] Source: {data.get('source', 'unknown')}")
        print(f"[INFO] Generated at: {data.get('timestamp', 'unknown')}")
        
        return recipe_ids
        
    except FileNotFoundError:
        print(f"[ERROR] Recipe IDs file not found: {RECIPE_IDS_FILE}")
        print("[INFO] Make sure to run images.ipynb first to generate the recipe IDs")
        return set()
    except Exception as e:
        print(f"[ERROR] Failed to load recipe IDs: {e}")
        return set()

# Load the recipe IDs that have images
recipe_ids_with_images = load_recipe_ids_with_images()

if not recipe_ids_with_images:
    print("[ERROR] No recipe IDs with images found. Cannot proceed with document filtering.")
    raise SystemExit("No recipe IDs with images found")


[INFO] Loaded 7 recipe IDs with images
[INFO] Source: s3://formatted-zone/images/
[INFO] Generated at: 2025-10-12T15-17-27Z


## 3. Multipart JSONL Writer for Large Files

The recipes dataset (`recipes.jsonl`) can be extremely large. Instead of loading everything into memory, we use a streaming approach with multipart uploads to handle the filtering efficiently.


In [8]:
MIN_PART_SIZE = 8 * 1024 * 1024  # 8 MB

class MultipartJSONLWriter:
    """Streaming JSONL writer that uses S3 multipart uploads for large files."""
    
    def __init__(self, bucket: str, key: str, content_type="application/x-ndjson", metadata=None):
        self.bucket = bucket
        self.key = key
        self.buf = io.BytesIO()
        self.parts = []
        self.part_num = 1
        self.open = True
        extra = {
            "Bucket": bucket,
            "Key": key,
            "ContentType": content_type,
            "Metadata": metadata or {},
        }
        resp = s3.create_multipart_upload(**extra)
        self.upload_id = resp["UploadId"]

    def _flush_part(self):
        """Flush the current buffer as a multipart upload part."""
        self.buf.seek(0)
        body = self.buf.read()
        if not body:
            self.buf.seek(0)
            self.buf.truncate(0)
            return
        resp = s3.upload_part(
            Bucket=self.bucket, Key=self.key,
            UploadId=self.upload_id, PartNumber=self.part_num, Body=body
        )
        self.parts.append({"ETag": resp["ETag"], "PartNumber": self.part_num})
        self.part_num += 1
        self.buf.seek(0); self.buf.truncate(0)

    def write_line(self, raw_line_bytes: bytes):
        """Write a line to the buffer, flushing when buffer is full."""
        # raw_line_bytes is already one JSON object line (no trailing \n required)
        self.buf.write(raw_line_bytes)
        self.buf.write(b"\n")
        if self.buf.tell() >= MIN_PART_SIZE:
            self._flush_part()

    def close(self):
        """Close the writer and complete the multipart upload."""
        if not self.open:
            return
        try:
            # If there's leftover data, flush as a last part
            self._flush_part()
            if not self.parts:
                # No data kept: abort multipart, optionally create empty object
                s3.abort_multipart_upload(
                    Bucket=self.bucket, Key=self.key, UploadId=self.upload_id
                )
                # Optional: write a 0-byte file so the path exists
                s3.put_object(
                    Bucket=self.bucket, Key=self.key, Body=b"",
                    ContentType="application/x-ndjson",
                    Metadata={"note": "empty after filtering", "ts": utc_ts()},
                )
            else:
                s3.complete_multipart_upload(
                    Bucket=self.bucket, Key=self.key, UploadId=self.upload_id,
                    MultipartUpload={"Parts": self.parts}
                )
        finally:
            self.open = False

def read_jsonl_lines(bucket: str, key: str):
    """Stream JSONL lines from S3."""
    obj = s3.get_object(Bucket=bucket, Key=key)
    for raw in obj["Body"].iter_lines():
        if raw:  # skip empty
            yield raw


## 4. Stream-Filter Documents by Recipe IDs

Now we filter the recipe documents to keep only those that have corresponding images. This step ensures that every recipe in the Trusted Zone can be paired with one or more valid images.


In [9]:
def filter_docs_to_trusted_by_ids():
    """Filter documents to keep only those with corresponding images."""
    total = kept = 0
    
    if DRY_RUN:
        print("[DRY_RUN] Counting documents that would be kept...")
        for raw in read_jsonl_lines(FORM_BUCKET, FORM_DOCS_KEY):
            total += 1
            try:
                rid = json.loads(raw).get("id")
            except Exception:
                continue
            if rid in recipe_ids_with_images:
                kept += 1
        print(f"[DRY_RUN] total={total} kept={kept}")
        return total, kept

    print("Filtering documents and uploading to Trusted Zone...")
    
    writer = MultipartJSONLWriter(
        TRUST_BUCKET, TRUST_DOCS_KEY,
        content_type="application/x-ndjson",
        metadata={"note": "filtered to ids that have images", "ts": utc_ts()}
    )
    
    try:
        for raw in read_jsonl_lines(FORM_BUCKET, FORM_DOCS_KEY):
            total += 1
            try:
                rec = json.loads(raw)
            except Exception:
                continue
            rid = rec.get("id")
            if rid and rid in recipe_ids_with_images:
                kept += 1
                writer.write_line(raw)
    finally:
        # Always close; it handles zero-kept gracefully
        writer.close()

    print(f"[OK] wrote filtered docs to s3://{TRUST_BUCKET}/{TRUST_DOCS_KEY}")
    return total, kept

# Execute the filtering
total_docs, kept_docs = filter_docs_to_trusted_by_ids()
print(f"[STATS] docs total={total_docs} kept={kept_docs} dropped={total_docs-kept_docs}")


Filtering documents and uploading to Trusted Zone...
[OK] wrote filtered docs to s3://trusted-zone/documents/recipes.jsonl
[STATS] docs total=1029720 kept=7 dropped=1029713


## 5. Generate Processing Report

Finally, we generate a comprehensive report of the document processing step and save it to the Trusted Zone for audit and monitoring purposes.


In [10]:
report = {
    "timestamp": utc_ts(),
    "processing_step": "documents",
    "source_docs": f"s3://{FORM_BUCKET}/{FORM_DOCS_KEY}",
    "destination_docs": f"s3://{TRUST_BUCKET}/{TRUST_DOCS_KEY}",
    "recipe_ids_source": RECIPE_IDS_FILE,
    "total_doc_count": total_docs,
    "kept_doc_count": kept_docs,
    "dropped_doc_count": total_docs - kept_docs,
    "unique_recipe_ids_with_images": len(recipe_ids_with_images),
    "filtering_rate": f"{(kept_docs/total_docs*100):.2f}%" if total_docs > 0 else "0%",
    "dry_run": DRY_RUN,
    "overwrite": OVERWRITE
}

if not DRY_RUN:
    s3.put_object(
        Bucket=TRUST_BUCKET,
        Key=f"{TRUST_REPORT_PREFIX}/documents_processing_{utc_ts()}.json",
        Body=json.dumps(report, ensure_ascii=False, indent=2).encode("utf-8"),
        ContentType="application/json"
    )
    print(f"[OK] wrote report -> s3://{TRUST_BUCKET}/{TRUST_REPORT_PREFIX}/")
else:
    print("[DRY_RUN] report:", json.dumps(report, indent=2))

print("\n" + "="*60)
print("DOCUMENTS PROCESSING COMPLETE")
print("="*60)
print(f"Filtered {kept_docs:,} documents from {total_docs:,} total")
print(f"Filtering rate: {report['filtering_rate']}")
print(f"Trusted Zone ready for analysis and modeling")


[OK] wrote report -> s3://trusted-zone/reports/

DOCUMENTS PROCESSING COMPLETE
Filtered 7 documents from 1,029,720 total
Filtering rate: 0.00%
Trusted Zone ready for analysis and modeling
